In [1]:
# Import required libraries
import pandas as pd
import dash
#import dash_html_components as html
#import dash_core_components as dcc
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [3]:
sites = spacex_df['Launch Site'].unique()
dropdown_list=[{'label':'All Sites', 'value':'All'}]
for site in sites:
    dropdown_list.append({'label':site, 'value':site})

In [ ]:

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                            options=dropdown_list,
                                            value='All',
                                            placeholder='Selecr a Launch Site here',
                                            searchable=True
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0:'0', 100: '100'},
                                                value= [min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output('success-pie-chart', 'figure'), Input('site-dropdown','value'))
def dispay_pie_chart(selected_site):
    if selected_site == 'All':
        fig = px.pie(spacex_df, values='class', names='Launch Site', title='All Site')
    else:
        filtered_df = spacex_df[spacex_df['Launch Site']==selected_site].groupby('class').count().reset_index()
        fig = px.pie(filtered_df, values='Launch Site', names='class', title=selected_site)
    return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output('success-payload-scatter-chart', 'figure'), [Input('site-dropdown', 'value'), Input('payload-slider', 'value')])
def scatter_chart(selected_site, payload_range):
    lo, hi = payload_range
    scatter_df = spacex_df[spacex_df['Payload Mass (kg)'].between(lo, hi)]
    if selected_site != 'All':
        scatter_df = scatter_df[scatter_df['Launch Site']==selected_site]
    fig = px.scatter(scatter_df, x='Payload Mass (kg)', y='class', color='Booster Version Category')
    return fig
# Run the app
if __name__ == '__main__':
    app.run_server()